In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import json
import random
import cv2
from sklearn.ensemble import RandomForestRegressor
from PIL import Image
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, Add
from tensorflow.keras import layers
import pickle

<b>Load Dataset from COCO Format

In [ ]:
# Define the Base Path of the Dataset
train_base_path = "E:/Deep Learning/ID Card/train"
valid_base_path = "E:/Deep Learning/ID Card/valid"

# Define the Path of the Tarining and Validaton Dataset
valid_path = os.path.join(valid_base_path, "Valid_annotation_coco.json")
train_path = os.path.join(train_base_path, "train_annotation_coco.json")

<b>Read the Valid/Train Dataset

In [ ]:
# Read the Valid Json File
file = open(valid_path)  # open the Json File

# Load the Data from Json file
valid_data = json.load(file) 

# Clos the Json File
file.close()

# Read the Train Json File
file = open(train_path)  # open the Json File

# Load the Data from Json file
train_data = json.load(file) 

# Clos the Json File
file.close()

In [ ]:
def load_data(data):
    # Get the Information of the Images 
    images_list = {}
    for image_data in data["images"]:
        image_name = image_data["file_name"]
        image_height = image_data["height"]
        image_width  = image_data["width"]
        image_id = image_data["id"]
        images_list[image_id] = [image_name , image_height , image_width]
    
    # Get the Information of the Category
    category_list = []
    for category in data["categories"]:
        category_list.append(category["name"])
    
    # Get the Information of the Images Annotations
    previous_image_id = ""
    annotation_list = {}
    count = 1
    for annotate in data["annotations"]:
        image_id = annotate["image_id"]
        category_id = annotate["category_id"]
        bbox = annotate["bbox"]

        if image_id == previous_image_id:
            annotation_list[image_id].append(category_id)
            annotation_list[image_id].append(bbox)
        else:
            annotation_list[image_id]   = [category_id , bbox]
            previous_image_id = image_id
    
    return images_list , category_list , annotation_list

In [ ]:
# Get the Information of Train Dataset
train_images , train_categories , train_annotations = load_data(train_data)

In [ ]:
# Get the Information of Validation Dataset
val_images   , val_categories   , val_annotations   = load_data(valid_data)

<b>Draw the Bounding Box in the Traning Images

In [ ]:
# Now Iterate the Images 

for index in range(len(train_images)):
    # Extract the Information about Images
    image_name   = train_images[index][0]
    image_height = train_images[index][1]
    image_width  = train_images[index][2]
    
    # Now Create the Path of the Image
    image_path = os.path.join(train_base_path , image_name)
    
    # Read the Image
    image = Image.open(image_path)
    
     # Convert the image into numpy array images
    cv2_image = np.array(image)
            
    # Extract the Information about Annotations of the image
    data_annotation = train_annotations[index]
    for ann_index in range(len(data_annotation)):
        if ann_index % 2 != 0:
            cv2.rectangle(cv2_image , (int(data_annotation[ann_index][0]) , int(data_annotation[ann_index][1])) , (int(data_annotation[ann_index][0])+int(data_annotation[ann_index][2]) , int(data_annotation[ann_index][1])+int(data_annotation[ann_index][3])) , (0,255,0) , 3)
    
    # Display the Image
    display(Image.fromarray(cv2_image))

<b>Preporcessing Train/Validation Images

In [ ]:
def list_data(images_data , image_annotations , base_path):
    # Define the List for Images and Annotations
    images_list     = []
    annotation_list = []

    # Iterate the Images
    for index in range(len(images_data)):
        # Extract the Information about Images
        image_name   = images_data[index][0]
        image_width  = images_data[index][2]
        image_height = images_data[index][1]

        # Now Create the Path of the Image
        image_path = os.path.join(base_path , image_name)

        # Read the Image
        image = Image.open(image_path)

         # Convert the image into numpy array images & Normalize the Image
        cv2_image = np.array(image).astype("float64") / 255.0
        resize_image = cv2.resize(cv2_image , (200,200))

        # Extract the Information about Annotations of the image
        data_annotation = image_annotations[index]
        annotation_sm = []
        for ann_index in range(len(data_annotation)):
            if ann_index % 2 != 0:
                # extract the Image BBOX
                x      = float(data_annotation[ann_index][0]) / image_width
                y      = float(data_annotation[ann_index][1]) / image_height
                width  = float(data_annotation[ann_index][2]) / image_width
                height = float(data_annotation[ann_index][3]) / image_height

                # Append the Image BBOX
                annotation_sm.append(x)
                annotation_sm.append(y)
                annotation_sm.append(width)
                annotation_sm.append(height)

        # Append the Images in the List
        images_list.append(resize_image)

        # Append the Image BBOX in the Annotations List
        annotation_list.append(annotation_sm)
        
    # Return the Results
    return images_list , annotation_list

Train Images & Annotations

In [ ]:
# Extract the Images and Annotation in the List
train_images_list , train_annotations_list = list_data(train_images , train_annotations , train_base_path)

# Display the Shape of the Images and Annotation
print(f"Shape of Images is     : {np.shape(train_images_list)}")
print(f"Shape of Annotation is : {np.shape(train_annotations_list)}")

In [ ]:
# Convert into the Numpy Array
train_images_np      = np.array(train_images_list)
train_annotations_np = np.array(train_annotations_list)

# Display the Shapes of  Images and Annotation
print(f"Shape of Images is     : {train_images_np.shape}")
print(f"Shape of Annotation is : {train_annotations_np.shape}")

Validation Images & Annotations

In [ ]:
# Extract the Images and Annotation in the List
val_images_list , val_annotations_list = list_data(val_images , val_annotations , valid_base_path)

# Display the Shape of the Images and Annotation
print(f"Shape of Images is     : {np.shape(val_images_list)}")
print(f"Shape of Annotation is : {np.shape(val_annotations_list)}")

In [ ]:
# Convert into the Numpy Array
val_images_np      = np.array(val_images_list)
val_annotations_np = np.array(val_annotations_list)

# Display the Shape of the Images and Annotation
print(f"Shape of Images is     : {val_images_np.shape}")
print(f"Shape of Annotation is : {val_annotations_np.shape}")

<b>Build CNN Feature Extraction Text Detection in the ID Card Image

In [ ]:
# Create the model of CNN 
feature_extractor = Sequential()

# Create the Convolutional Layers
feature_extractor.add(Conv2D(32, kernel_size=(9, 9), activation='relu', input_shape=(200, 200, 3)))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(96, kernel_size=(7, 7), activation='relu' ))
feature_extractor.add(BatchNormalization())


feature_extractor.add(MaxPooling2D(pool_size=(3, 3) , strides=(3, 3)))

feature_extractor.add(Conv2D(128, kernel_size=(3,3) , strides=(2,2) , padding="valid"))
feature_extractor.add(BatchNormalization())

feature_extractor.add(Conv2D(filters=1024, kernel_size=(3, 3), strides=(3,3) ,  padding='valid'))
feature_extractor.add(BatchNormalization())
feature_extractor.add(MaxPooling2D())

# Flatten the Feature Extractor
feature_extractor.add(Flatten())

# Display the Summary of the Feature Extractor
feature_extractor.summary()

<b>Text Detection Random Forest Regressor

In [ ]:
# Create the Obect of Random Forest Classifier
rf = RandomForestRegressor(n_estimators = 1000 , random_state = 42)

In [ ]:
# Create the Dataset with Features Extractor 
x_rf_data = feature_extractor.predict(train_images_np)

# Display the Shape of the X_RF_DATA
print(f"Shape of the X Data for RF is : {x_rf_data.shape}")

In [ ]:
# Fit the Random Forest Classifier in Training Dataset
rf.fit(x_rf_data , train_annotations_np)

In [ ]:
# Save the Ranfom Forest Regressor
with open('C:/Users/User131022/Desktop/New folder/model.pkl', 'wb') as file:
    pickle.dump(rf, file)

In [ ]:
# Load the Random Forest Regressor
with open('C:/Users/User131022/Desktop/New folder/model.pkl', 'rb') as file:
    model = pickle.load(file)

<b>Predict the Result

In [ ]:
# Get the image path
image_path = "E:\\Deep Learning\\ID Card\\test\\1.jpg"

# Read the Image
image = Image.open(image_path)

# Get the Width & Height of the Image
w,h = image.size

# Resize the Image
resie_image = image.resize((200,200))

# Convert the Image into Numpy Array
cv2_image = np.array(resie_image).astype("float64") / 255.0

In [ ]:
# Normalize the Image
image_batch = np.expand_dims(cv2_image, axis=0)

# Get the Fetuare of the Image
feature_image = feature_extractor.predict(image_batch)

# Predict the Images
prediction = model.predict(feature_image)[0]

# Convert the again Numpy Array
res_image = np.array(image)

In [ ]:
count = 0
for index in range(len(prediction)):
    x = int(prediction[count] * w)
    count += 1
    y = int(prediction[count] * h)
    count += 1
    width = int(prediction[count] * w)
    count += 1
    height = int(prediction[count] * h)
    cv2.rectangle(res_image, (x,y) , (x+width , y+height) , (255,0,0) , 3)
    count += 1
    if count == len(prediction):
        break

In [ ]:
display(Image.fromarray(res_image))